# Google Cloud CMIP6 Public Data: Basic Python Example

This is actually borrowed from the [CMIP6 Cookbook](https://projectpythia.org/cmip6-cookbook) maintained by [Project Pythia](https://projectpythia.org).

The notebook was adapted from an earlier example in the [Pangeo gallery](http://gallery.pangeo.io/repos/pangeo-gallery/cmip6/basic_search_and_load.html).

---

## Overview

This notebooks shows how to query the Google Cloud CMIP6 catalog and load the data using Python.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |

- **Time to learn**: 10 minutes

---

## Imports

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6


## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

The columns of the dataframe correspond to the [CMI6 controlled vocabulary](https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q/edit).

Here we filter the data to find monthly surface air temperature for historical experiments.

In [ ]:
df_ta = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'historical'")
df_ta

Now we do further filtering to find just the models from NCAR.

In [ ]:
df_ta_ncar = df_ta.query('institution_id == "NCAR"')
df_ta_ncar

## Load Data

Now we will load a single store using `fsspec`, `zarr`, and `xarray`.

In [ ]:
# get the path to a specific zarr store (the last one from the dataframe above)
zstore = df_ta_ncar.zstore.values[-1]
print(zstore)

# create a mutable-mapping-style interface to the store
mapper = fsspec.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True)
ds

Plot a map from a specific date:

In [ ]:
myfield = ds.tas.sel(time='1950-01').load()
myfield.squeeze().plot()

The global mean of a lat-lon field needs to be weighted by the area of each grid cell, which is proportional to cosine of latitude:

In [ ]:
def global_mean(field):
    weights = np.cos(np.deg2rad(field.lat))
    return field.weighted(weights).mean(dim=('lat','lon'))

Applying this to a single time slice yields a single scalar temperature.

Let's express the result in degrees Celsius rather than Kelvin:

In [ ]:
global_mean(myfield) - 273.15

Create a timeseries of global-average surface air temperature. 

We can just pass all the temperature data through our `global_mean` function:

In [ ]:
ta_timeseries = global_mean(ds.tas)
ta_timeseries

By default the data are loaded lazily, as Dask arrays. Here we trigger computation explicitly.

In [ ]:
%time ta_timeseries.load()

Now let's make a plot!

In [ ]:
ta_timeseries.plot(label='monthly')
ta_timeseries.rolling(time=12).mean().plot(label='12 month rolling mean')
plt.legend()
plt.title('Global Mean Surface Air Temperature')

---

## Summary
In this notebook, we opened a CESM2 dataset with `fsspec` and `zarr`. We calculated and plotted global average surface air temperature. 

## Resources and references
- [Original notebook in the Pangeo Gallery](http://gallery.pangeo.io/repos/pangeo-gallery/cmip6/basic_search_and_load.html) by Henri Drake and [Ryan Abernathey](https://ocean-transport.github.io/)
- [The CMIP6 Cookbook](https://projectpythia.org/cmip6-cookbook/) (Project Pythia)